# Кратчайшие расстояния во взвешенных графах

Дан взешенный граф. Заданы стартовая s и конечная t веришины. Необходимо найти кратчайшйи путь из s в t. Для решения этой задачи и некоторых похожих на нее задач существует ряд алгоритмов, которые дальше будут рассмотрены. Сразу обусловим некоторые обозначения. Неизвестные расстояния будем помечать как бесконечность. Конечно, тут можно использовать `float("inf")`, просто заведомо большое число, `None` или `-1`  (при условии, что в задаче не возможны отрицательные расстояния). Здесь для конкретности будет использована флотовая бесконечность `float("inf")`. Для нее определены все арифметические операции и числовые сравнения, поэтому писать какой-то специальный код для работы с ней не понадобится. Считаем, что в графе $n$ вершин и $m$ ребер.

## Алгоритм Дейкстры (Dijkstra)

Алгоритм Дейкстры возволяет искать кратчайшие расстояния от одной вершины до всех и работает следующим образом:

+ на каждой итерации алгоритм среди непомеченных вершин выбирает вершину с наименьшим до нее расстоянием;
+ помечает вершину как посещенную;
+ пытается улучшить расстояние до смежных с ней вершин.

На каждой итерации поддерживается инвариант, что расстояния до помеченных вершин являются кратчайшими и более меняться не будут. Однако, чтобы это условие не нарушалось, граф не должен содержать ребер отрицательного веса. Иначе, алгоритм в такой задаче не применим. Код алгоритма выглядит следующим образом:

In [ ]:
if __name__ == "__main__":
    # Считываем граф, преобразуем его в список смежности, который храним в adj_list
    d = [float("inf")]*n
    d[s] = 0
    used == ["False"] * n
    while True:
        u = -1
        for i in range(n):
            if not used[i] and (u == -1 or d[i] < d[u]):
                u = i
        if u == -1 or d[u] == float("inf)": 
            break
        used[u] = True
        for v, w in adj_list[u]:
            d[v] = min(d[v], d[u] + w)

Время работы алгоритма зависит от того, как быстро ищется минимум. В приведенном выше варианте время работы $O(n^2)$. Для ускорения алгоритма применяют кучу либо дерево отрезков. Для обоих структур справедливо, что их высота порядка $O(\log{n})$, а значит и операции с ними работают за это время. Благодаря им операция извлечения минимума работает за $O(\log{n})$ вместо $O(n)$. Но при этом возрастает время работы опреации обновления расстояния до вершины: с $O(1)$ до $O(\log{n})$. ПОлучается, что мы не более, чем n раз выполняем операцию извлечения минимума. Не более, чем m раз мы обновляем расстояние до вершины. Таким образом исходная асимптотика $O(n^2 + m) = O(n^2)$ меняется на $O(n \log{n} + m \log{n}) = O((n + m) \log{n})$.

Ниже приведен пример реализации алгоритма Дейкстры с кучей. Заметим, что сама по себе опреация добавления элемента в кучу нам не нужна, ее мы можем построить просто при помощи list comprehension. Но у нас появилась новая операция, изменение приоритета у элемента, которую в общем случае куча не поддерживает. Для того, чтобы добавить поддержку операции, необходимо знать местоположение элемента в куче. Выполнять поиск элемента за линию просто ухудшит изначальную асимптотику. Поэтому заведем массив `v2h`, в котором будет поддерживаться актуальное положение элементов в куче. И при изменении кучи меняется и он.

In [ ]:
def sift_up(heap, v2h, i):
    while i > 0 and heap[(i - 1) // 2]["dist"] > heap[i]["dist"]:
        v2h[heap[i]["vert"]], v2h[heap[(i - 1) // 2]["vert"]] = (i - 1) // 2, i
        heap[i], heap[(i - 1) // 2] = heap[(i - 1) // 2], heap[i]
        i = (i - 1) // 2


def sift_down(heap, v2h, i):
    n = len(heap)
    while i * 2 + 1 < n:
        j = i
        if heap[i]["dist"] > heap[i * 2 + 1]["dist"]:
            j = i * 2 + 1
        if i * 2 + 2 < n and heap[j]["dist"] > heap[i * 2 + 2]["dist"]:
            j = i * 2 + 2
        if i == j:
            break
        v2h[heap[i]["vert"]], v2h[heap[j]["vert"]] = j, i
        heap[i], heap[j] = heap[j], heap[i]
        i = j


def extract_min(heap, v2h):
    x = heap[0]["dist"]
    heap[0] = heap.pop()
    v2h[heap[0]["vert"]] = 0
    sift_down(heap, v2h, 0)
    return x


def entry(vert, dist):
    return {
        "vert": vert,
        "dist": dist
    }


if __name__ == "__main__":
    # Считываем граф, преобразуем его в список смежности, который храним в adj_list
    d = [float("inf")] * n
    d[s] = 0
    heap = [entry(0, s)] + [entry(float("inf"), i) for i in range(n) if i != s]
    v2h = [0] * n
    for i, j in enumerate(heap):
        v2h[j["vert"]] = i
    while heap:
        u = extract_min(heap, v2h)
        if d[u] == float("inf"):
            break
        for v, w in adj_list[u]:
            if d[v] > d[u] + w:
                d[v] = d[u] + w
                heap[v2h[v]]["dist"] = d[v]
                sift_up(heap, v2h, v2h[v])
    print(d[t])

## Алгоритм Форда-Беллмана (Ford-Bellman)

Алгоритм Форда-Беллмана, как и алгоритм Дейкстры, используется для поиска кратчайшего расстояния от одной вершины до остальных. Он является типичным алгоритмом ДП. Состояния описываются двумя параметрами и означают "длину кратчайшего пути, проходящего не более, чем по i ребрам, и заканчивающегося в вершине j".

In [ ]:
if __name__ == "__main__":
    # Cчитываем граф, преобразуем его в список ребер, который храним в edges
    d = [float("inf")]*n
    d[s] = 0
    for _ in range(n-1):
        for u, v, w in edges:
            if d[u] is not None:
                d[v] = min(INF if d[v] is None else d[v], d[u] + w)

Такой алгоритм работает $O(n m)$. Заметим несколько вещей:

1. aлгоритм работает корректно даже при наличии ребер отрицательного веса, `-1` - валидное значение для расстояний, поэтому массив d инициализировать `-1` нельзя;
2. вернувшись в вершину, пройдя по циклу, расстояние до нее не может уменьшится (циклы отрицательного веса пока не рассматриваем);
3. исходя из (2) для нахождения кратчайшего пути до всех вершин достаточно $N-1$ итерации, т.е. кратчайшие пути до всех вершин не содержат циклов.

Однако утверждение (2) справедливо, только когда нет циклов отрицательного веса, т.е. цикла, в которой растояния до вершин в нем будут каждый раз уменьшаться, если мы будем по нему гулять. Таким образом нам вообще не выгодно его заканчивать, а значит мы можем считать, что кратчайшие расстояния до этих вершин будут $-\infty$. Таким образом $N-1$ итерации не хватит чтобы посчитать кратчайшие расстояния. Поэтому мы можем внешний цикл увеличить на одну итерацию. Все вершины, расстояние до которых обновится на последней итерации, можем считать имеют расстояние $-\infty$.

Отсюда можно сделать вывод, что алгоритм применяется не только для поиска кратчайших расстояний в графе, но и для поиска циклов отрицательного веса.